In [2]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf

tf.__version__

'1.5.0'

In [ ]:
class HierarchicalAttentionNetwork(object):
    def __init__(self, vocab_size, num_class, embed_dim, hidden_dim):
        super(HierarchicalNeuralNetwork, self).__init__()
        self._vocab_size = vocab_size
        self._num_class = num_class
        self._embed_dim = embed_dim
        self._hidden_dim = hidden_dim
    def _make_graph(self, graph):
        with graph.as_default():
            words  = tf.placeholder(tf.int32, [None, None], name='words')
            length = tf.placeholder(tf.int32, [None], name='length')
            labels = tf.placeholder(tf.int32, (), name='labels') 
            
            with tf.variable_scope('embeddings'):
                embedding = tf.get_variable('lookup', shape=(self._vocab_size, self._embed_dim), dtype=tf.float32, trainable=True)
                embeded   = tf.nn.embedding_lookup(embedding, words)
            with tf.variable_scope('words_lstm'):
                cell_fw = tf.nn.rnn_cell.GRUCell(num_units=self._hidden_dim)
                cell_bw = tf.nn.rnn_cell.GRUCell(num_units=self._hidden_dim)
                (outputs_fw, outputs_bw), _ = \
                    tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, embeded, sequence_length=length)
            outputs = tf.concat([outputs_fw, outputs_bw], axis=2)
            with tf.variable_scope('words_attention'):
                hidden = tf.layers.dense(outputs, units=self._hidden_dim * 2, activation=tf.nn.tanh)
                attention = tf.layers.dense(outputs, units=1, activation=tf.nn.softmax)
            sentence_embedding = tf.expand_dims(outputs * attention, axis=0)
            with tf.variable_scope('sentence_lstm'):
                cell_fw = tf.nn.rnn_cell.GRUCell(num_units=self._hidden_dim)
                cell_bw = tf.nn.rnn_cell.GRUCell(num_units=self._hidden_dim)
                (outputs_fw, outputs_bw), _ = \
                    tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, sentence_embedding)
            outputs = tf.squeeze(tf.concat([outputs_fw, outputs_bw], axis=2), axis=[0])
            with tf.variable_scope('sentence_attention'):
                hidden = tf.layers.dense(outputs, units=self._hidden_dim * 2, activation=tf.nn.tanh)
                attention = tf.layers.dense(hidden, units=1, activation=tf.nn.softmax)
            outputs = outputs * attention
            logits = tf.layers.dense(outputs, units=self._num_class, activation=None)
            loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
            training_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)
            return words, length, labels, logits, loss, training_op
        def fit(self, words_seq, length_seq, labels_seq, num_epochs=10, model=None):
            graph = tf.Graph()
            words, length, labels, _, loss, training_op = self._make_graph(graph)
            
            with tf.Session(graph=graph) as sess:
                sess.run(tf.global_variables_initializer())
                for epoch in range(num_epochs):
                    for words_val, length_val, labels_val in zip(words_seq, length_seq, labels_seq):
                        feed_dict = { words : words_val, length : length_val, labels : labels_val }
                        sess.run(training_op, feed_dict=feed_dict)
                    loss_total = 0
                    for words_val, length_val, labels_val in zip(words_seq, length_seq, labels_seq):
                        feed_dict = { words : words_val, length : length_val, labels : labels_val }
                        loss_val = sess.run(loss, feed_dict=feed_dict)
                        loss_total += loss_val
                    print('Epoch [%d/%d], Loss: %.3f' % (epoch + 1, num_epochs, loss_total))